In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, count

In [2]:
spark = SparkSession.builder.master('local[*]').appName('used-car-price').config('spark.executor.memory', '6g').getOrCreate()
spark = SparkSession.builder.master('local[*]').appName('used-car-price')\
.config("spark.executor.instances", "1")\
.config("spark.executor.memory", "6g")\
.config("spark.driver.memory", "6g")\
.config("spark.executor.memoryOverhead", "8g")\
.getOrCreate()

24/09/11 13:18:23 WARN Utils: Your hostname, langchain resolves to a loopback address: 127.0.1.1; using 192.168.0.103 instead (on interface wlp3s0)
24/09/11 13:18:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/11 13:18:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/11 13:18:25 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
csv_file_path = '../data/cleaned_train'
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

In [4]:
df.show()

+--------------------+--------------------+-----+-------------+----------+------+-------------+--------------------+--------------------+----------+---------------+-----------+-----+
|               model|              engine|   id|        brand|model_year|milage|    fuel_type|        transmission|             ext_col|   int_col|       accident|clean_title|price|
+--------------------+--------------------+-----+-------------+----------+------+-------------+--------------------+--------------------+----------+---------------+-----------+-----+
|   Malibu Limited LT|2520HP 36L V6 Cyl...|28616|    Chevrolet|      2006| 53000|     Gasoline|           automatic|              Silver|      Gray|  None reported|        Yes|15000|
|S-Class S 580 4MATIC|      40 Liter Turbo|28617|Mercedes-Benz|      2022| 11238|       Hybrid|           automatic|Anthracite Blue M...|     Black|  None reported|       NULL|49998|
|       1500 Big Horn|3950HP 57L 8 Cyli...|28618|          RAM|      2021| 11806|    

In [5]:
data = df.drop('id')

In [6]:
data.show()

+--------------------+--------------------+-------------+----------+------+-------------+--------------------+--------------------+----------+---------------+-----------+-----+
|               model|              engine|        brand|model_year|milage|    fuel_type|        transmission|             ext_col|   int_col|       accident|clean_title|price|
+--------------------+--------------------+-------------+----------+------+-------------+--------------------+--------------------+----------+---------------+-----------+-----+
|   Malibu Limited LT|2520HP 36L V6 Cyl...|    Chevrolet|      2006| 53000|     Gasoline|           automatic|              Silver|      Gray|  None reported|        Yes|15000|
|S-Class S 580 4MATIC|      40 Liter Turbo|Mercedes-Benz|      2022| 11238|       Hybrid|           automatic|Anthracite Blue M...|     Black|  None reported|       NULL|49998|
|       1500 Big Horn|3950HP 57L 8 Cyli...|          RAM|      2021| 11806|     Gasoline|   8-speed automatic|     

In [7]:
from sklearn.impute import KNNImputer
import pandas as pd
def knn_impute(df, n_neighbors=5):   
    df_encoded = df.copy()
    for col in df_encoded.select_dtypes(include='object').columns:
        df_encoded[col] = df_encoded[col].astype('category').cat.codes
    knn_imputer = KNNImputer(n_neighbors=n_neighbors)
    df_imputed = pd.DataFrame(knn_imputer.fit_transform(df_encoded), columns=df_encoded.columns)
    for col in df.select_dtypes(include='object').columns:
        df_imputed[col] = df_imputed[col].round().astype(int).map(
            dict(enumerate(df[col].astype('category').cat.categories)))
    return df_imputed

In [8]:
df_train_imputed = knn_impute(data.toPandas(), n_neighbors=25)

In [9]:
from sklearn.preprocessing import OrdinalEncoder
cat_cols_train = df_train_imputed.select_dtypes(include=['object']).columns
cat_cols_train = cat_cols_train[cat_cols_train != 'class']
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

df_train_imputed[cat_cols_train] = ordinal_encoder.fit_transform(df_train_imputed[cat_cols_train].astype(str))


In [10]:
df_train_imputed.isnull().sum()

model           0
engine          0
brand           0
model_year      0
milage          0
fuel_type       0
transmission    0
ext_col         0
int_col         0
accident        0
clean_title     0
price           0
dtype: int64

In [11]:
Sp_data = spark.createDataFrame(df_train_imputed)

In [12]:
Sp_data.show()

+------+------+-----+----------+--------+---------+------------+-------+-------+--------+-----------+-------+
| model|engine|brand|model_year|  milage|fuel_type|transmission|ext_col|int_col|accident|clean_title|  price|
+------+------+-----+----------+--------+---------+------------+-------+-------+--------+-----------+-------+
|1020.0| 332.0|  9.0|    2006.0| 53000.0|      2.0|        23.0|  262.0|   71.0|     0.0|        0.0|15000.0|
|1433.0| 768.0| 36.0|    2022.0| 11238.0|      3.0|        23.0|    6.0|   14.0|     0.0|        1.0|49998.0|
|   4.0| 761.0| 44.0|    2021.0| 11806.0|      2.0|        18.0|  303.0|   10.0|     0.0|        0.0|68000.0|
| 687.0| 966.0| 21.0|    2018.0| 29000.0|      2.0|        18.0|  127.0|   31.0|     0.0|        0.0|62000.0|
|1107.0| 231.0| 14.0|    2005.0|156350.0|      2.0|        23.0|  303.0|   71.0|     0.0|        0.0| 4400.0|
| 659.0| 428.0| 14.0|    2021.0|  8332.0|      2.0|         1.0|  127.0|   21.0|     0.0|        0.0|31650.0|
|1089.0| 8

In [13]:
Sp_data.columns

['model',
 'engine',
 'brand',
 'model_year',
 'milage',
 'fuel_type',
 'transmission',
 'ext_col',
 'int_col',
 'accident',
 'clean_title',
 'price']

In [14]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.functions import vector_to_array
from pyspark.sql.functions import col

feature_columns = [column for column in Sp_data.columns if column != "price"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
assembled_df = assembler.transform(Sp_data)
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)
scaler_model = scaler.fit(assembled_df)
scaled_df = scaler_model.transform(assembled_df)
scaled_df = scaled_df.withColumn("scaled_features_array", vector_to_array(scaled_df["scaled_features"]))
num_features = len(feature_columns)
for i in range(num_features):
    scaled_df = scaled_df.withColumn(f"scaled_feature_{i+1}", col("scaled_features_array")[i])

scaled_df_to_save = scaled_df.select([f"scaled_feature_{i+1}" for i in range(num_features)] + ["price"])
scaled_df_to_save.write.csv("../data/scaled_training_data", header=True)
scaler_model.write().overwrite().save("scaler_model")


24/09/11 13:18:40 WARN TaskSetManager: Stage 6 contains a task of very large size (1008 KiB). The maximum recommended task size is 1000 KiB.
24/09/11 13:18:41 WARN TaskSetManager: Stage 9 contains a task of very large size (1008 KiB). The maximum recommended task size is 1000 KiB.


In [15]:
scaled_training_data_pandas = scaled_df_to_save.toPandas()
scaled_training_data_pandas.head()

24/09/11 13:18:44 WARN TaskSetManager: Stage 14 contains a task of very large size (1008 KiB). The maximum recommended task size is 1000 KiB.


,scaled_feature_1,scaled_feature_2,scaled_feature_3,scaled_feature_4,scaled_feature_5,scaled_feature_6,scaled_feature_7,scaled_feature_8,scaled_feature_9,scaled_feature_10,scaled_feature_11,price
0,0.313392,-0.914874,-0.885279,-1.732270,-0.296328,-0.100413,0.678382,0.918100,0.917805,-0.548562,-0.347229,15000.0
1,1.116338,0.698395,0.819098,1.131814,-1.194192,1.314961,0.678382,-1.319286,-0.522527,-0.548562,2.879920,49998.0
2,-1.661895,0.672494,1.324098,0.952808,-1.181981,-0.100413,0.035275,1.276431,-0.623603,-0.548562,-0.347229,68000.0
3,-0.334020,1.431027,-0.127778,0.415793,-0.812317,-0.100413,0.035275,-0.261772,-0.092954,-0.548562,-0.347229,62000.0
4,0.482535,-1.288590,-0.569653,-1.911275,1.925651,-0.100413,0.678382,1.276431,0.917805,-0.548562,-0.347229,4400.0
